In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import pm4py

In [4]:
event_log = pm4py.read_xes('BPI_Challenge_2013_incidents.xes.gz')
event_log = event_log.sort_values(['case:concept:name','time:timestamp'])
number_of_traces = event_log['case:concept:name'].nunique()
subset_el = event_log[['case:concept:name','concept:name','time:timestamp','org:resource','org:role']]
subset_el['org:role'] = subset_el['org:role'].fillna('nan_1').apply(lambda x: x.split('_')[0])
subset_el['org:role'] = subset_el['org:role'].replace({'C':'C1','D':'D1','E':'E1'})

parsing log, completed traces :: 100%|██████████| 7554/7554 [00:03<00:00, 2246.04it/s]
/tmp/ipykernel_11728/4067086581.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_el['org:role'] = subset_el['org:role'].fillna('nan_1').apply(lambda x: x.split('_')[0])
/tmp/ipykernel_11728/4067086581.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_el['org:role'] = subset_el['org:role'].replace({'C':'C1','D':'D1','E':'E1'})


In [11]:
import pandas as pd
from pm4py.objects.log.util import dataframe_utils

def rename_repeating_events(df, epsilon=3, final_states=None):
    df = dataframe_utils.convert_timestamp_columns_in_df(df)
    df = df.sort_values(by=["case:concept:name", "time:timestamp"])

    updated_final_states = set()

    def process_case(case_df):
        new_names = []
        event_counts = {}
        for name in case_df["concept:name"]:
            if name not in event_counts:
                event_counts[name] = 1
            else:
                event_counts[name] += 1

            count = event_counts[name]
            if count <= epsilon:
                new_name = name if count == 1 else f"{name}{count}"
            else:
                new_name = f"{name}{epsilon}"

            new_names.append(new_name)

        case_df["concept:name"] = new_names

        if final_states:
            last_event = new_names[-1]
            original_last_event = case_df["concept:name"].iloc[-1].split(str(epsilon))[0].rstrip("1234567890")
            if original_last_event in final_states:
                updated_final_states.add(last_event)

        return case_df

    df = df.groupby("case:concept:name", group_keys=False).apply(process_case)

    if final_states is not None:
        return df, sorted(updated_final_states)
    return df


In [12]:
final_states = ['Completed']
new_df, new_final_states  = rename_repeating_events(subset_el, 3, final_states)
new_df

/home/vco/.pyenv/versions/pn-updater/lib/python3.11/site-packages/pm4py/objects/log/util/dataframe_utils.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = strpfromiso.fix_dataframe_column(df[col])
/tmp/ipykernel_11728/1227030263.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("case:concept:name", group_keys=False).apply(process_case)


,case:concept:name,concept:name,time:timestamp,org:resource,org:role
0,1-364285768,Accepted,2010-03-31 16:59:42+00:00,Frederic,A2
1,1-364285768,Accepted2,2010-03-31 17:00:56+00:00,Frederic,A2
2,1-364285768,Queued,2010-03-31 17:45:48+00:00,Frederic,A2
3,1-364285768,Accepted3,2010-04-06 16:44:07+00:00,Anne Claire,A2
4,1-364285768,Queued2,2010-04-06 16:44:38+00:00,Anne Claire,A2
...,...,...,...,...,...
65528,1-740866691,Completed,2012-05-05 04:07:43+00:00,Lierson,V3
65529,1-740866708,Accepted,2012-05-05 06:58:40+00:00,Lierson,V3
65530,1-740866708,Accepted2,2012-05-05 07:02:39+00:00,Lierson,V3
65531,1-740866708,Completed,2012-05-05 07:23:03+00:00,Lierson,V3


In [13]:
new_final_states

['Completed', 'Completed2', 'Completed3']